In [1]:
!pip install websocket-client
!pip install ibm-cos-sdk
! pip install tweepy
! pip install textblob

import pandas as pd
import json, types
import websocket
import _thread
import time
import ibm_boto3
import codecs
import tweepy #https://github.com/tweepy/tweepy
import numpy as np
import re
import matplotlib.pyplot as plt

from io import StringIO
from textblob import TextBlob


#  Twitter API credentials
consumer_key = "consumer_key"
consumer_secret = "consumer_secret"
access_key = "access_key"
access_secret = "access_secret"
OAUTH_KEYS = {'consumer_key':consumer_key, 'consumer_secret':consumer_secret,
 'access_token_key':access_key, 'access_token_secret':access_secret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


     |████████████████████████████████| 636 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.1 MB/s eta 0:00:011
     |████████████████████████████████| 660 kB 18.7 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 8.6 MB/s  eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434678 sha256=e9ff420c00dffdefc415127fea32c895cea2ef8c1c87771bb6dd58434acdf611
  Stored in directory: /home/jovyan/.cache/pip/wheels/de/5e/42/64abaeca668161c3e2cecc24f864a8fc421e3d07a104fc8a51
Successfully built nltk


In [2]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = auth
        self.twitter_client = api

        self.twitter_user = twitter_user
    
    #Return twetter api
    def get_twitter_client_api(self):
        return self.twitter_client
    
    #Returns the num_tweets most recent statuses by specific user, including retweets, posted by the authenticating user and that user’s friends.
    def get_user_timeline_tweets(self, num_tweets,hashtag):
        tweets = []
        for tweet in tweepy.Cursor(self.twitter_client.user_timeline, q=hashtag,id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
            #print(tweet)
        return tweets
    
     #Returns the num_tweets most recent statuses by specific user, including retweets, posted by the authenticating user and that user’s friends.
    def get_hashtag_timeline_tweets(self, num_tweets,hashtag):
        tweets = []
        for tweet in tweepy.Cursor(self.twitter_client.search, q=hashtag).items(num_tweets):
            tweets.append(tweet)
            #print(tweet)
        return tweets


In [3]:
class TweetAnalyzer():
    def __init__(self, twitter_client):
        self.twitter_client = twitter_client
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1
    
    #All possible attribute
    """
    ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', 
    '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', 
    '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', 
    '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'entities', 'favorite', 'favorite_count', 'favorited', 
    'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 
    'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'parse', 'parse_list', 'place', 'possibly_sensitive', 'retweet', 'retweet_count', 
    'retweeted', 'retweets', 'source', 'source_url', 'text', 'truncated', 'user']
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        #length of text
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        #number of retweets
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        
        #All possible attribute of a user
        """
        'created_at', 'default_profile', 'default_profile_image', 'description', 'entities', 'favourites_count', 'follow', 
        'follow_request_sent', 'followers', 'followers_count', 'followers_ids', 'following', 'friends', 'friends_count', 'geo_enabled', 
        'has_extended_profile', 'id', 'id_str', 'is_translation_enabled', 'is_translator', 'lang', 'listed_count', 'lists', 
        'lists_memberships', 'lists_subscriptions', 'location', 'name'
        """
         
        user_data = [tweet.user for tweet in tweets]
        df['name'] = np.array([user_tweet.name for user_tweet in user_data])
        df['description'] = np.array([user_tweet.description for user_tweet in user_data])
        df['location'] = np.array([user_tweet.location for user_tweet in user_data])
        df['created_at'] = np.array([user_tweet.created_at for user_tweet in user_data])
            
        df['followers_count'] = np.array([user_tweet.followers_count for user_tweet in user_data])
        df['friends_count'] = np.array([user_tweet.friends_count for user_tweet in user_data])
        df['following'] = np.array([user_tweet.following for user_tweet in user_data])
            
            

        return df

In [4]:

def on_message(ws, message):
    print(message)
    msg = json.loads(message)
    cmd = msg['cmd']
    if cmd == 'MINING':
        user = msg['user']
        
        numTweet = int(msg['numTweet'])
        hashtag = msg['hashtag']
        twitter_client = TwitterClient(user)
        tweet_analyzer = TweetAnalyzer(twitter_client)
        #Tweets_filter
        tweets = twitter_client.get_user_timeline_tweets(numTweet,hashtag)

        #From json to DataFrame
        df = tweet_analyzer.tweets_to_data_frame(tweets)

        #Sentiment Analysis
        df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])
        out = df.to_json(orient='records')
        #print(df.head(numTweet))
        responsejson = {}
        responsejson['forcmd']='MINING'
        responsejson['response']= out
        ws.send(json.dumps(responsejson))
    if cmd == 'MININGHASHTAG':
        numTweet = int(msg['numTweet'])
        hashtag = msg['hashtag']
        twitter_client = TwitterClient()
        tweet_analyzer = TweetAnalyzer(twitter_client)
        #Tweets_filter
        tweets = twitter_client.get_hashtag_timeline_tweets(numTweet,hashtag)

        #From json to DataFrame
        df = tweet_analyzer.tweets_to_data_frame(tweets)

        #Sentiment Analysis
        df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])
        out = df.to_json(orient='records')
        #print(df.head(numTweet))
        responsejson = {}
        responsejson['forcmd']='MININGHASHTAG'
        responsejson['response']= out
        ws.send(json.dumps(responsejson))
    

def on_error(ws, error):
    print(error)

def on_close(ws):
    ws.send("MiningTwitter BackEnd Closed")

def on_open(ws):
    def run(*args):
        for i in range(10000):
            hbeat = '{"cmd":"MiningTwitter BackEnd Connected"}'
            ws.send(hbeat)
            time.sleep(100)
            
    _thread.start_new_thread(run, ())


def start_websocket_listener():
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("ws://<NODERED_BASE_URL>/ws/orchestrate",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

In [5]:
start_websocket_listener()

send: b'\x81\x9c\x9b\x9b\xcf\t\xd6\xf2\xa1`\xf5\xfc\x9b~\xf2\xef\xbbl\xe9\xbb\x8dh\xf8\xf0\x8ag\xff\xbb\x8ce\xf4\xe8\xaam'
error from callback <function on_close at 0x7f7fb09572f0>: socket is already closed.


[Errno -2] Name or service not known


  File "/srv/conda/envs/notebook/lib/python3.6/site-packages/websocket/_app.py", line 346, in _callback
    callback(self, *args)
  File "<ipython-input-4-8cab2b99b802>", line 51, in on_close
    ws.send("MiningTwitter BackEnd Closed")
  File "/srv/conda/envs/notebook/lib/python3.6/site-packages/websocket/_app.py", line 154, in send
    if not self.sock or self.sock.send(data, opcode) == 0:
  File "/srv/conda/envs/notebook/lib/python3.6/site-packages/websocket/_core.py", line 253, in send
    return self.send_frame(frame)
  File "/srv/conda/envs/notebook/lib/python3.6/site-packages/websocket/_core.py", line 279, in send_frame
    l = self._send(data)
  File "/srv/conda/envs/notebook/lib/python3.6/site-packages/websocket/_core.py", line 449, in _send
    return send(self.sock, data)
  File "/srv/conda/envs/notebook/lib/python3.6/site-packages/websocket/_socket.py", line 135, in send
    raise WebSocketConnectionClosedException("socket is already closed.")
